Конвертер

In [18]:
#пример инпута DE
text ="""ivo :: na –k
auto :: n
je :: =proxi +k t
ε :: =f proxi
kupio :: =n na= v
pravo :: adv
ε :: =n n –focal
ε :: =proxi +focal f
ε :: =v +focal f
ε :: =v proxi
ε :: =t c
-ε :: =t c1
-ε :: =c1 +foc fc
ε :: =n n –foc
ε :: =c s
ε :: =fc s
u :: =n p
ε :: =pp pp –foc
seo :: =pp na= v
ε :: =p adv= p
ε :: =p pp
u :: p1
ε :: p1= =adj adjp –foc
ε :: =n adjp= p
ε :: =n adj= n
ε :: =adj adj –foc
lijepi :: adj"""


# Разделение входного текста на строки
grammar_lines = text.split('\n')

transformed_rules = []

# Обработка каждого правила из исходной грамматики
for rule_line in grammar_lines:
    # Разделение строки на левую (LHS) и правую (RHS) части относительно '::'
    original_lhs, original_rhs = rule_line.split('::')

    # Создание обратного правила: RHS становится LHS и наоборот, с добавлением скобок
    # Например, для "A :: B" получим "(B) -> A"
    reversed_rule = "(" + original_rhs[1:] + ")" + ' -> ' + original_lhs[:-1]

    # Добавление преобразованного правила в список
    transformed_rules.append(reversed_rule)
    print(reversed_rule)  # Отладочный вывод

# Функция для объединения правил
def merge_rules(rule_list):
    added_count = 0  # Счетчик добавленных правил
    # Перебор комбинаций правил
    for i in range(len(rule_list)):
        for j in range(i+1, len(rule_list)):
            # Извлечение первого символа из LHS первого правила
            first_symbol_1 = rule_list[i].split('->')[0][1:-2].split()[0]
            # Извлечение первого символа из LHS второго правила
            first_symbol_2 = rule_list[j].split('->')[0][1:-2].split()[0]

            # Удаление запятой в конце символов, если есть
            if first_symbol_1.endswith(','):
                first_symbol_1 = first_symbol_1[:-1]
            if first_symbol_2.endswith(','):
                first_symbol_2 = first_symbol_2[:-1]

            # Сравнение длин символов для обработки случаев с '='
            swapped = False
            if len(first_symbol_1) > len(first_symbol_2):
                first_symbol_1, first_symbol_2 = first_symbol_2, first_symbol_1
                swapped = True  # Флаг обмена местами

            # Проверка условий для слияния:
            # Если один символ начинается/заканчивается '=', а другой совпадает с основной частью
            if (first_symbol_2.startswith("=") and first_symbol_1 == first_symbol_2[1:]) or (first_symbol_2.endswith("=") and first_symbol_1 == first_symbol_2[:-1]):
                # Извлечение RHS частей (после преобразования)
                rule1_rhs = rule_list[i].split('->')[0][:-1]
                rule2_rhs = rule_list[j].split('->')[0][:-1]

                # Восстановление порядка, если было перестановка
                if not swapped:
                    rule1_rhs, rule2_rhs = rule2_rhs, rule1_rhs

                # Объединение RHS частей
                combined_rhs = f"{rule1_rhs} {rule2_rhs}"
                # Обработка LHS: удаление первого символа и лишних символов
                processed_lhs1 = (" ".join(rule1_rhs.split()[1:]))[:-1]
                processed_lhs2 = (" ".join(rule2_rhs.split()[1:]))[:-1]

                # Разделение элементов LHS с учетом запятых и пробелов
                lhs_elements1 = [e.strip() for e in (processed_lhs1.split(',') if ',' in processed_lhs1 else processed_lhs1.split())]
                lhs_elements2 = [e.strip() for e in (processed_lhs2.split(',') if ',' in processed_lhs2 else processed_lhs2.split())]

                # Проверка на наличие конфликтующих признаков (начинающихся с '–'),реализация SMC
                has_conflict = True
                common_elements = set(lhs_elements1) & set(lhs_elements2)
                for element in common_elements:
                    if element.startswith('–'):
                        has_conflict = False  # Конфликт разрешим

                # Формирование нового объединенного LHS

                new_lhs = f"({processed_lhs1}, {processed_lhs2})" if processed_lhs2 else f"({processed_lhs1})"
                new_rule = new_lhs + " -> " + combined_rhs

                # Добавление нового правила, если его нет в списке и нет конфликта
                if new_rule not in rule_list and has_conflict:
                    rule_list.append(new_rule)
                    print(new_rule)  # Отладочный вывод
                    added_count += 1
    return added_count  # Возврат количества добавленных правил

# Функция для удаления пар признаков (+x и –x) из LHS
def remove_feature_pairs(rule_list):
    for idx in range(len(rule_list)):
        original_rhs_side = rule_list[idx].split('->')[0]
        if ',' not in original_rhs_side:
            continue  # Пропуск правил без запятых (нет пар)

        # Обработка LHS: удаление скобок и разделение элементов
        current_lhs = original_rhs_side[1:].strip()
        current_lhs = current_lhs[:-1].strip().split()
        first_feature = current_lhs[0]
        matching_index = -1

        # Поиск парного признака (начинается с '–' и совпадает по имени)
        for feature_idx in range(1, len(current_lhs)):
            current_feature = current_lhs[feature_idx].rstrip(',')
            if first_feature.startswith('+') and current_feature.startswith('–') and first_feature[1:] == current_feature[1:]:
                matching_index = feature_idx
                break

        # Если пара найдена, создаем упрощенное правило
        if matching_index != -1:
            # Удаление пары признаков из LHS
            before_feature = " ".join(current_lhs[1:matching_index])
            after_feature = " ".join(current_lhs[matching_index+1:])
            simplified_lhs = f"({(before_feature + ' ' + after_feature).strip()})"
            new_rule = simplified_lhs + ' -> ' + original_rhs_side

            # Добавление нового правила, если его нет в списке
            if new_rule not in rule_list:
                rule_list.append(new_rule)
                print(new_rule)  # Отладочный вывод

# Основной цикл преобразований: объединение и удаление пар до тех пор, пока есть изменения
added_count = merge_rules(transformed_rules)
while added_count:
    added_count = merge_rules(transformed_rules)
    remove_feature_pairs(transformed_rules)

# Вывод общего количества преобразованных правил
print(len(transformed_rules))

(na –k) -> ivo
(n) -> auto
(=proxi +k t) -> je
(=f proxi) -> ε
(=n na= v) -> kupio
(adv) -> pravo
(=n n –focal) -> ε
(=proxi +focal f) -> ε
(=v +focal f) -> ε
(=v proxi) -> ε
(=t c) -> ε
(=t c1) -> -ε
(=c1 +foc fc) -> -ε
(=n n –foc) -> ε
(=c s) -> ε
(=fc s) -> ε
(=n p) -> u
(=pp pp –foc) -> ε
(=pp na= v) -> seo
(=p adv= p) -> ε
(=p pp) -> ε
(p1) -> u
(p1= =adj adjp –foc) -> ε
(=n adjp= p) -> ε
(=n adj= n) -> ε
(=adj adj –foc) -> ε
(adj) -> lijepi
(na= v) -> (=n na= v) (n)
(n –focal) -> (=n n –focal) (n)
(n –foc) -> (=n n –foc) (n)
(p) -> (=n p) (n)
(adjp= p) -> (=n adjp= p) (n)
(adj= n) -> (=n adj= n) (n)
(na= v, –focal) -> (=n na= v) (n –focal)
(na= v, –foc) -> (=n na= v) (n –foc)
(n –focal, –foc) -> (=n n –focal) (n –foc)
(n –foc, –focal) -> (=n n –foc) (n –focal)
(p, –focal) -> (=n p) (n –focal)
(p, –foc) -> (=n p) (n –foc)
(p, –focal, –foc) -> (=n p) (n –focal, –foc)
(p, –foc, –focal) -> (=n p) (n –foc, –focal)
(adv= p) -> (=p adv= p) (p)
(adv= p, –focal) -> (=p adv= p) (p, –focal)

Генератор без рекурсии, генерирует только часть необходимых последовательностей

In [14]:
#пример инпута
rules_text="""
(na –k) -> ivo
(n) -> auto
(=proxi +k t) -> je
(=f proxi) -> ε
(=n na= v) -> kupio
(adv) -> pravo
(=n n –focal) -> ε
(=proxi +focal f) -> ε
(=v +focal f) -> ε
(=v proxi) -> ε
(=t c) -> ε
(=t c1) -> -ε
(=c1 +foc fc) -> -ε
(=n n –foc) -> ε
(=c s) -> ε
(=fc s) -> ε
(=n p) -> u
(=pp pp –foc) -> ε
(=pp na= v) -> seo
(=p adv= p) -> ε
(=p pp) -> ε
(p1) -> u
(p1= =adj adjp –foc) -> ε
(=n adjp= p) -> ε
(=n adj= n) -> ε
(=adj adj –foc) -> ε
(adj) -> lijepi
(na= v) -> (=n na= v) (n)
(n –focal) -> (=n n –focal) (n)
(n –foc) -> (=n n –foc) (n)
(p) -> (=n p) (n)
(adjp= p) -> (=n adjp= p) (n)
(adj= n) -> (=n adj= n) (n)
(na= v, –focal) -> (=n na= v) (n –focal)
(na= v, –foc) -> (=n na= v) (n –foc)
(n –focal, –foc) -> (=n n –focal) (n –foc)
(n –foc, –focal) -> (=n n –foc) (n –focal)
(p, –focal) -> (=n p) (n –focal)
(p, –foc) -> (=n p) (n –foc)
(p, –focal, –foc) -> (=n p) (n –focal, –foc)
(p, –foc, –focal) -> (=n p) (n –foc, –focal)
(adv= p) -> (=p adv= p) (p)
(adv= p, –focal) -> (=p adv= p) (p, –focal)
(adv= p, –foc) -> (=p adv= p) (p, –foc)
(adv= p, –focal, –foc) -> (=p adv= p) (p, –focal, –foc)
(adv= p, –foc, –focal) -> (=p adv= p) (p, –foc, –focal)
(pp) -> (=p pp) (p)
(pp, –focal) -> (=p pp) (p, –focal)
(pp, –foc) -> (=p pp) (p, –foc)
(pp, –focal, –foc) -> (=p pp) (p, –focal, –foc)
(pp, –foc, –focal) -> (=p pp) (p, –foc, –focal)
(=adj adjp –foc) -> (p1= =adj adjp –foc) (p1)
(adjp= p, –focal) -> (=n adjp= p) (n –focal)
(adjp= p, –foc) -> (=n adjp= p) (n –foc)
(adjp= p, –focal, –foc) -> (=n adjp= p) (n –focal, –foc)
(adjp= p, –foc, –focal) -> (=n adjp= p) (n –foc, –focal)
(adj= n, –focal) -> (=n adj= n) (n –focal)
(adj= n, –foc) -> (=n adj= n) (n –foc)
(adj= n, –focal, –foc) -> (=n adj= n) (n –focal, –foc)
(adj= n, –foc, –focal) -> (=n adj= n) (n –foc, –focal)
(adj –foc) -> (=adj adj –foc) (adj)
(n) -> (adj= n) (adj)
(adjp –foc) -> (=adj adjp –foc) (adj)
(n, –focal) -> (adj= n, –focal) (adj)
(n, –foc) -> (adj= n, –foc) (adj)
(n, –focal, –foc) -> (adj= n, –focal, –foc) (adj)
(n, –foc, –focal) -> (adj= n, –foc, –focal) (adj)
(v, –k) -> (na= v) (na –k)
(v, –focal, –k) -> (na= v, –focal) (na –k)
(v, –foc, –k) -> (na= v, –foc) (na –k)
(na= v, –focal, –foc) -> (=n na= v) (n –focal, –foc)
(na= v, –foc, –focal) -> (=n na= v) (n –foc, –focal)
(na= v, –focal) -> (=n na= v) (n, –focal)
(na= v, –foc) -> (=n na= v) (n, –foc)
(na= v, –focal, –foc) -> (=n na= v) (n, –focal, –foc)
(na= v, –foc, –focal) -> (=n na= v) (n, –foc, –focal)
(p) -> (adv= p) (adv)
(p, –focal) -> (adv= p, –focal) (adv)
(p, –foc) -> (adv= p, –foc) (adv)
(p, –focal, –foc) -> (adv= p, –focal, –foc) (adv)
(p, –foc, –focal) -> (adv= p, –foc, –focal) (adv)
(n –focal, –foc) -> (=n n –focal) (n, –foc)
(+focal f, –k) -> (=v +focal f) (v, –k)
(+focal f, –focal, –k) -> (=v +focal f) (v, –focal, –k)
(+focal f, –foc, –k) -> (=v +focal f) (v, –foc, –k)
(proxi, –k) -> (=v proxi) (v, –k)
(proxi, –focal, –k) -> (=v proxi) (v, –focal, –k)
(proxi, –foc, –k) -> (=v proxi) (v, –foc, –k)
(n –foc, –focal) -> (=n n –foc) (n, –focal)
(p, –focal) -> (=n p) (n, –focal)
(p, –foc) -> (=n p) (n, –foc)
(p, –focal, –foc) -> (=n p) (n, –focal, –foc)
(p, –foc, –focal) -> (=n p) (n, –foc, –focal)
(pp –foc) -> (=pp pp –foc) (pp)
(pp –foc, –focal) -> (=pp pp –foc) (pp, –focal)
(na= v) -> (=pp na= v) (pp)
(na= v, –focal) -> (=pp na= v) (pp, –focal)
(na= v, –foc) -> (=pp na= v) (pp, –foc)
(na= v, –focal, –foc) -> (=pp na= v) (pp, –focal, –foc)
(na= v, –foc, –focal) -> (=pp na= v) (pp, –foc, –focal)
(na= v, –foc) -> (=pp na= v) (pp –foc)
(na= v, –foc, –focal) -> (=pp na= v) (pp –foc, –focal)
(adjp= p, –focal) -> (=n adjp= p) (n, –focal)
(adjp= p, –foc) -> (=n adjp= p) (n, –foc)
(adjp= p, –focal, –foc) -> (=n adjp= p) (n, –focal, –foc)
(adjp= p, –foc, –focal) -> (=n adjp= p) (n, –foc, –focal)
(adj= n, –focal) -> (=n adj= n) (n, –focal)
(adj= n, –foc) -> (=n adj= n) (n, –foc)
(adj= n, –focal, –foc) -> (=n adj= n) (n, –focal, –foc)
(adj= n, –foc, –focal) -> (=n adj= n) (n, –foc, –focal)
(p, –foc) -> (adjp= p) (adjp –foc)
(n, –foc) -> (adj= n) (adj –foc)
(p, –focal, –foc) -> (adjp= p, –focal) (adjp –foc)
(n, –focal, –foc) -> (adj= n, –focal) (adj –foc)
(f, –k) -> (+focal f, –focal, –k)
(v, –focal, –foc, –k) -> (na= v, –focal, –foc) (na –k)
(v, –foc, –focal, –k) -> (na= v, –foc, –focal) (na –k)
(+k t, –k) -> (=proxi +k t) (proxi, –k)
(+k t, –focal, –k) -> (=proxi +k t) (proxi, –focal, –k)
(+k t, –foc, –k) -> (=proxi +k t) (proxi, –foc, –k)
(proxi, –k) -> (=f proxi) (f, –k)
(+focal f, –k) -> (=proxi +focal f) (proxi, –k)
(+focal f, –focal, –k) -> (=proxi +focal f) (proxi, –focal, –k)
(+focal f, –foc, –k) -> (=proxi +focal f) (proxi, –foc, –k)
(+focal f, –focal, –foc, –k) -> (=v +focal f) (v, –focal, –foc, –k)
(+focal f, –foc, –focal, –k) -> (=v +focal f) (v, –foc, –focal, –k)
(proxi, –focal, –foc, –k) -> (=v proxi) (v, –focal, –foc, –k)
(proxi, –foc, –focal, –k) -> (=v proxi) (v, –foc, –focal, –k)
(t,) -> (+k t, –k)
(t, –focal,) -> (+k t, –focal, –k)
(t, –foc,) -> (+k t, –foc, –k)
(f, –foc, –k) -> (+focal f, –focal, –foc, –k)
(f, –foc, –k) -> (+focal f, –foc, –focal, –k)
(+k t, –focal, –foc, –k) -> (=proxi +k t) (proxi, –focal, –foc, –k)
(+k t, –foc, –focal, –k) -> (=proxi +k t) (proxi, –foc, –focal, –k)
(proxi, –foc, –k) -> (=f proxi) (f, –foc, –k)
(+focal f, –focal, –foc, –k) -> (=proxi +focal f) (proxi, –focal, –foc, –k)
(+focal f, –foc, –focal, –k) -> (=proxi +focal f) (proxi, –foc, –focal, –k)
(c) -> (=t c) (t,)
(c, –focal,) -> (=t c) (t, –focal,)
(c, –foc,) -> (=t c) (t, –foc,)
(c1) -> (=t c1) (t,)
(c1, –focal,) -> (=t c1) (t, –focal,)
(c1, –foc,) -> (=t c1) (t, –foc,)
(+foc fc) -> (=c1 +foc fc) (c1)
(+foc fc, –focal,) -> (=c1 +foc fc) (c1, –focal,)
(+foc fc, –foc,) -> (=c1 +foc fc) (c1, –foc,)
(s) -> (=c s) (c)
(s, –focal,) -> (=c s) (c, –focal,)
(s, –foc,) -> (=c s) (c, –foc,)
(t, –focal, –foc,) -> (+k t, –focal, –foc, –k)
(t, –foc, –focal,) -> (+k t, –foc, –focal, –k)
(fc,) -> (+foc fc, –foc,)
(c, –focal, –foc,) -> (=t c) (t, –focal, –foc,)
(c, –foc, –focal,) -> (=t c) (t, –foc, –focal,)
(c1, –focal, –foc,) -> (=t c1) (t, –focal, –foc,)
(c1, –foc, –focal,) -> (=t c1) (t, –foc, –focal,)
(+foc fc, –focal, –foc,) -> (=c1 +foc fc) (c1, –focal, –foc,)
(+foc fc, –foc, –focal,) -> (=c1 +foc fc) (c1, –foc, –focal,)
(s, –focal, –foc,) -> (=c s) (c, –focal, –foc,)
(s, –foc, –focal,) -> (=c s) (c, –foc, –focal,)
(s) -> (=fc s) (fc,)
(fc, –focal,) -> (+foc fc, –focal, –foc,)
(fc, –focal,) -> (+foc fc, –foc, –focal,)
(s, –focal,) -> (=fc s) (fc, –focal,)
"""

In [17]:
import re
import math
from collections import defaultdict

class ParseTreeGenerator:
    def __init__(self, rules_text):
        self.rules = self.parse_rules(rules_text)
        self.rule_weights = self.calculate_rule_weights()

    def parse_rules(self, rules_text): #Обработка правил, группировка по левой части
        rules = defaultdict(list) #Сохраняем в словарь
        for line in rules_text.strip().split("\n"):
            left, right = line.split(" -> ", 1) #Разбиваем на левую,правую части
            right_parts = tuple(re.findall(r"\([^)]+\)|\S+", right))  #Скобки тоже элементы
            rules[left].append(right_parts)
        return dict(rules)

    def calculate_rule_weights(self): #Подсчет весов(log2) для правил
        rule_weights = {}
        for left, productions in self.rules.items():
            count = len(productions) #Количество продолжений
            weight = math.log2(count) if count > 1 else 0
            for production in productions:
                rule_weights[(left, production)] = weight #записываем веса
        return rule_weights

    def generate_trees(self, symbol, path=None): #Генерируем псевдодеревья
        if path is None:
            path = set()
        if symbol in path:
            return []  # В этом варианте запрещаем рекурсию вообще
        path.add(symbol)

        if symbol not in self.rules:
            return [{"node": symbol, "log2": 0, "children": [], "total_log2": 0}]

        trees = []
        for production in self.rules[symbol]:
            sub_trees = [self.generate_trees(sym, path.copy()) for sym in production]
            tree_combinations = self.cartesian_product(*sub_trees) #Декартово произведение поддеревьев

            for combination in tree_combinations: #Сопоставление весов и поддеревьев
                weight = self.rule_weights.get((symbol, production), 0)
                total_weight = weight + sum(sub.get("total_log2", 0) for sub in combination) #Считаем стоимости для поддеревьев
                trees.append({          #Добавляем в псевдодеревья символ узла, правило с символом в левой части, вес правила, суммарный вес поддерева
                    "node": symbol,
                    "rule": f"{symbol} -> {' '.join(production)}" if production else f"{symbol} -> ε",
                    "log2": weight,
                    "children": combination,
                    "total_log2": total_weight
                })
        return trees

    def cartesian_product(self, *args): #декартово произведение поддеревьев
        if not args:
            return [[]]
        result = [[]]
        for group in args:
            temp = []
            for res in result:
                for item in group:
                    temp.append(res + [item])
            result = temp
        return result

    def print_tree(self, tree, indent=""): #Вывод псевдодеревьев
        if not tree:
            return
        if "rule" in tree:
            print(f"{indent}{tree['rule']}  [log2={tree['log2']:.2f}], Σlog2={tree['total_log2']:.2f}") #Для правил пишем стоимость собственную и поддерева
        else:
            print(f"{indent}{tree['node']}") #терминальный узел
        for child in tree["children"]: #запись цепочек соотвествующего поддерева
            self.print_tree(child, indent + "  ")

    def display_trees(self, trees):
        for i, tree in enumerate(trees):
            self.print_tree(tree)
            print("\n" + "#" * 80 + "\n") #Разделитель между псевдодеревьями
a = 0
for i,tree in enumerate(trees):
  a = a+1
print(a)
generator = ParseTreeGenerator(rules_text)
trees = generator.generate_trees("(s)") #Задаем начальный символ для генерации
generator.display_trees(trees)


70
(s) -> (=c s) (c)  [log2=1.00], Σlog2=4.00
  (=c s) -> ε  [log2=0.00], Σlog2=0.00
    ε
  (c) -> (=t c) (t,)  [log2=0.00], Σlog2=3.00
    (=t c) -> ε  [log2=0.00], Σlog2=0.00
      ε
    (t,) -> (+k t, –k)  [log2=0.00], Σlog2=3.00
      (+k t, –k) -> (=proxi +k t) (proxi, –k)  [log2=0.00], Σlog2=3.00
        (=proxi +k t) -> je  [log2=0.00], Σlog2=0.00
          je
        (proxi, –k) -> (=v proxi) (v, –k)  [log2=1.00], Σlog2=3.00
          (=v proxi) -> ε  [log2=0.00], Σlog2=0.00
            ε
          (v, –k) -> (na= v) (na –k)  [log2=0.00], Σlog2=2.00
            (na= v) -> (=n na= v) (n)  [log2=1.00], Σlog2=2.00
              (=n na= v) -> kupio  [log2=0.00], Σlog2=0.00
                kupio
              (n) -> auto  [log2=1.00], Σlog2=1.00
                auto
            (na –k) -> ivo  [log2=0.00], Σlog2=0.00
              ivo

################################################################################

(s) -> (=c s) (c)  [log2=1.00], Σlog2=5.00
  (=c s) -> ε  [log2=0.